In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve
from sklearn.inspection import permutation_importance

# Configuration pour un affichage plus propre
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

C:\Users\ethan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 1. Chargement
X = pd.read_csv("../Data/X.csv")
y = pd.read_csv("../Data/y.csv")

# 2. Séparation Train / Test
# Stratify est crucial pour garder la même proportion de départs dans le train et le test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3. Nettoyage des colonnes inutiles (ID)
if 'id_employee' in X_train.columns:
    X_train = X_train.drop(columns=['id_employee'])
    X_test = X_test.drop(columns=['id_employee'])

print(f"Train shape : {X_train.shape}")
print(f"Test shape  : {X_test.shape}")

FileNotFoundError: [Errno 2] No such file or directory: '../Data/X.csv'

In [ ]:
# 1. Calcul du ratio de déséquilibre (Négatifs / Positifs)
count_0 = np.sum(y_train.values == 0)
count_1 = np.sum(y_train.values == 1)
ratio_theorique = float(count_0 / count_1)

print(f"Ratio naturel (0/1) : {ratio_theorique:.2f}")

# 2. Le Modèle de base
xgb = XGBClassifier(
    objective='binary:logistic',
    random_state=42,
    n_jobs=-1,
    subsample=0.8,         # Évite le sur-apprentissage (utilise 80% des données par arbre)
    colsample_bytree=0.8   # Évite le sur-apprentissage (utilise 80% des colonnes par arbre)
)

# 3. La GRANDE Grille
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],      # Le "pas" d'apprentissage
    'max_depth': [3, 5, 7, 10],              # Profondeur (XGBoost préfère souvent 3-7)
    'min_child_weight': [1, 3, 5],           # Poids minimum pour garder une feuille
    'scale_pos_weight': [ratio_theorique, ratio_theorique * 1.5] # On teste d'être plus agressif
}

# 4. Setup du GridSearch
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='f1',  # On maximise le F1-score (équilibre Précision/Rappel)
    n_jobs=-1,
    verbose=1
)

In [ ]:
print("Lancement de l'optimisation XGBoost...")
grid_search.fit(X_train, y_train.values.ravel())

print("Terminé !")

In [ ]:
# Meilleurs paramètres
print(f"Meilleurs paramètres : {grid_search.best_params_}")
print(f"Meilleur Score F1 (CV Train) : {grid_search.best_score_:.3f}")

best_xgb = grid_search.best_estimator_

# Analyse de la stabilité
cv_results = pd.DataFrame(grid_search.cv_results_)
best_idx = grid_search.best_index_
mean_score = cv_results.loc[best_idx, 'mean_test_score']
std_score = cv_results.loc[best_idx, 'std_test_score']

print(f"Validation Croisée (5 folds) : {mean_score:.3f} (+/- {std_score:.3f})")

if std_score > 0.05:
    print("⚠️ Attention : Le modèle est instable (forte variance).")
else:
    print("✅ Le modèle est stable.")

In [ ]:
# 1. Récupérer les probabilités sur le TEST set
y_probs = best_xgb.predict_proba(X_test)[:, 1]

# 2. Calculer la courbe Precision-Recall
precision, recall, thresholds = precision_recall_curve(y_test, y_probs)

# 3. Calcul du F1-score pour chaque seuil (Avec sécurité division par zéro)
numerator = 2 * precision * recall
denominator = precision + recall
fscore = np.divide(numerator, denominator, out=np.zeros_like(denominator), where=denominator!=0)

# 4. Trouver le meilleur seuil
ix = np.argmax(fscore)
best_thresh = thresholds[ix]

print(f"Meilleur Seuil (Threshold) = {best_thresh:.3f}")
print(f"F-Score Optimal = {fscore[ix]:.3f}")

# 5. Graphique
plt.figure(figsize=(8, 5))
plt.plot(recall, precision, label='XGBoost')
plt.scatter(recall[ix], precision[ix], marker='o', color='black', label='Optimal', s=100, zorder=5)
plt.xlabel('Rappel (Recall)')
plt.ylabel('Précision')
plt.title('Courbe Précision-Rappel & Point Optimal')
plt.legend()
plt.show()

In [ ]:
# Application du seuil optimisé
y_pred_optimal = (y_probs >= best_thresh).astype(int)

print("\n=== RAPPORT FINAL (TEST SET) ===")
print(classification_report(y_test, y_pred_optimal))

print("\n=== Matrice de Confusion ===")
cm = confusion_matrix(y_test, y_pred_optimal)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Prédiction')
plt.ylabel('Réalité')
plt.title(f'Matrice avec seuil {best_thresh:.3f}')
plt.show()

In [ ]:
# Initialisation JS
shap.initjs()

print("Calcul des valeurs SHAP...")
# Avec XGBoost, TreeExplainer est très efficace
explainer = shap.TreeExplainer(best_xgb)
shap_values = explainer(X_test)

# --- Summary Plot (Global) ---
# Visualisation des variables les plus impactantes
plt.figure()
shap.plots.beeswarm(shap_values, max_display=15, show=False)
plt.title("Impact global des variables (XGBoost)")
plt.show()

# --- Comparaison Importances ---
# On compare ce que le modèle dit (Feature Imp) vs la réalité mathématique (SHAP)
shap_importance = pd.DataFrame(np.abs(shap_values.values).mean(axis=0), index=X_test.columns, columns=['SHAP'])
xgb_importance = pd.DataFrame(best_xgb.feature_importances_, index=X_test.columns, columns=['XGBoost_Gain'])

comparison = pd.concat([shap_importance, xgb_importance], axis=1)
comparison = comparison.sort_values(by='SHAP', ascending=False).head(10)

print("\n=== Top 10 Variables ===")
display(comparison)

In [ ]:
# Récupération des indices
y_test_array = y_test.values.ravel()
# Trouver un vrai démissionnaire que le modèle a bien détecté (Vrai Positif)
true_positives = np.where((y_test_array == 1) & (y_pred_optimal == 1))[0]

if len(true_positives) > 0:
    idx = true_positives[0] # On prend le premier
    print(f"--- Analyse du Démissionnaire (Index Test: {idx}) ---")
    print(f"Probabilité prédite : {y_probs[idx]:.3f}")
    # Waterfall plot
    shap.plots.waterfall(shap_values[idx])
else:
    print("Aucun démissionnaire correctement détecté pour l'exemple.")